# Bird Image Classifier Training Script
### This notebook downloads bird images, trains a model using FastAI, and allows predictions.

In [ ]:
# Imports
import os
from pathlib import Path
from fastai.vision.all import *
from duckduckgo_search import DDGS
from fastcore.all import *

In [ ]:
# Check if running on Kaggle
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search  # Install required libraries on Kaggle

## Step 1: Download Images of Birds

In [ ]:
def search_images(term, max_images=30):
    """
    Search and download image URLs using DuckDuckGo.

    Args:
        term (str): Search term.
        max_images (int): Maximum number of images to return.

    Returns:
        List[str]: List of image URLs.
    """
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
        results = ddgs.images(term, max_results=max_images)
    return L(results).itemgot('image')

In [ ]:
# List of bird species
bird_species = [
    "Northern Cardinal", "Blue Jay", "American Crow", "Black-capped Chickadee",
    "White-breasted Nuthatch", "Mourning Dove", "House Sparrow", "House Finch",
    "Downy Woodpecker", "Red-bellied Woodpecker", "Tufted Titmouse", "Carolina Wren",
    "Eastern Screech-Owl", "Great Horned Owl", "American Robin", "Barn Swallow",
    "Tree Swallow", "Baltimore Oriole", "Eastern Bluebird", "Ruby-throated Hummingbird",
    "Red-winged Blackbird", "Common Yellowthroat", "Indigo Bunting", "Eastern Meadowlark",
    "Canada Goose", "Snow Goose", "Sandhill Crane", "Yellow Warbler", "Warbling Vireo",
    "Swainson's Thrush", "Northern Parula", "Black-throated Green Warbler", "Hermit Thrush",
    "Rose-breasted Grosbeak", "Dark-eyed Junco", "American Tree Sparrow", "Snow Bunting",
    "Rough-legged Hawk", "Common Redpoll", "Pine Siskin", "Mallard", "Wood Duck",
    "Great Blue Heron", "Killdeer", "Belted Kingfisher", "Spotted Sandpiper",
    "Double-crested Cormorant", "Bald Eagle", "Red-tailed Hawk", "Cooper's Hawk",
    "Sharp-shinned Hawk", "Peregrine Falcon", "American Kestrel", "Osprey",
    "Northern Flicker", "Pileated Woodpecker", "Cedar Waxwing", "Yellow-bellied Sapsucker",
    "Scarlet Tanager", "Northern Mockingbird", "Brown Thrasher", "Orchard Oriole",
    "Eastern Phoebe", "Great Crested Flycatcher"
]

In [ ]:
# Directory to save images
path = Path('birds')

# Download and preprocess bird images
for species in bird_species:
    dest = path / species
    dest.mkdir(exist_ok=True, parents=True)
    urls = search_images(f'{species} bird photo', max_images=100)
    download_images(dest, urls=urls)
    resize_images(dest, max_size=400, dest=dest)

## Step 2: Train the Model

In [ ]:
# Remove corrupted images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
print(f"Number of failed images removed: {len(failed)}")

In [ ]:
# DataBlock for the bird dataset
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

# Show a batch of images
dls.show_batch(max_n=6)

In [ ]:
# Train the model using ResNet18
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Test the Model

In [ ]:
# Test with a sample image
test_image = '../Birds Examples/Killdeer.jpg' 
predicted_species, _, probs = learn.predict(PILImage.create(test_image))
print(f"This is a: {predicted_species}.")
print(f"Probability it's a {predicted_species}: {probs.max():.4f}")